## Analyze e2e latencies for the single_client_e2e experiment

In [ ]:
import numpy as np
import glob

def get_append_metrics(path):
    file_pattern = path + "append_metrics*.csv"

    total_throughput = 0
    latency_values = []

    for file in glob.glob(file_pattern):
        with open(file, 'r') as f:
            lines = f.readlines()[1:]
            for line in lines:
                parts = line.strip().split(',')
                gsn, latency, throughput = int(parts[0]), float(parts[1]), float(parts[2])
                latency_values.append(latency)
            
            total_throughput += throughput

    latency_array = np.array(latency_values)

    mean_latency = np.mean(latency_array)
    p50_latency = np.percentile(latency_array, 50)
    p99_latency = np.percentile(latency_array, 99)

    print("results for computation time " + path.split("_")[-1].split("/")[0] + " us")
    print("statistic/metric, latency (us)")
    print(f"mean, {mean_latency:.2f}")
    print(f"p50, {p50_latency:.2f}")
    print(f"p99, {p99_latency:.2f}")
    print(f"total throughput, {total_throughput:.2f}")

    return int(path.split("_")[-1].split("/")[0]), mean_latency, total_throughput 

def get_e2e_metrics(path):
    file_pattern = path + "e2e_metrics*.csv"

    delivery_latency_values = []
    compute_latency_values = []
    confirmation_latency_values = []
    e2e_latency_values = []
    queuing_delay_values = []

    for file in glob.glob(file_pattern):
        with open(file, 'r') as f:
            lines = f.readlines()[1:]
            for line in lines:
                parts = line.strip().split(',')
                delivery, confirm, compute, e2e, queuing_delay = float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4]), float(parts[5])
                delivery_latency_values.append(delivery)
                compute_latency_values.append(compute)
                confirmation_latency_values.append(confirm)
                e2e_latency_values.append(e2e)
                queuing_delay_values.append(queuing_delay)

    
    delivery_latency_array = np.array(delivery_latency_values)
    compute_latency_array = np.array(compute_latency_values)
    confirmation_latency_array = np.array(confirmation_latency_values)
    e2e_latency_array = np.array(e2e_latency_values)
    queuing_delay_array = np.array([x for x in queuing_delay_values if x > 0])

    print("statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)")
    print(f"mean, {np.mean(delivery_latency_array):.2f}, {np.mean(compute_latency_array):.2f}, {np.mean(confirmation_latency_array):.2f}, {np.mean(e2e_latency_array):.2f}, {np.mean(queuing_delay_array):.2f}")
    print(f"std, {np.std(delivery_latency_array):.2f}, {np.std(compute_latency_array):.2f}, {np.std(confirmation_latency_array):.2f}, {np.std(e2e_latency_array):.2f}, {np.std(queuing_delay_array):.2f}")
    print(f"p50, {np.percentile(delivery_latency_array, 50):.2f}, {np.percentile(compute_latency_array, 50):.2f}, {np.percentile(confirmation_latency_array, 50):.2f}, {np.percentile(e2e_latency_array, 50):.2f}, {np.percentile(queuing_delay_array, 50):.2f}")
    print(f"p99, {np.percentile(delivery_latency_array, 99):.2f}, {np.percentile(compute_latency_array, 99):.2f}, {np.percentile(confirmation_latency_array, 99):.2f}, {np.percentile(e2e_latency_array, 99):.2f}, {np.percentile(queuing_delay_array, 99):.2f}")

    return int(path.split("_")[-1].split("/")[0]), np.mean(delivery_latency_array), np.mean(compute_latency_array), np.mean(confirmation_latency_array), np.mean(e2e_latency_array), np.mean(queuing_delay_array)

In [ ]:
path = "../results/*"

for dir in glob.glob(path + "*/"):
    get_append_metrics(dir)
    get_e2e_metrics(dir)


In [ ]:
import re
import pandas as pd

# Input data
data = """
PASTE DATA FROM OUTPUT ABOVE
"""

# Split data into lines
lines = data.splitlines()

# Parsing logic
results = []
current_time = None
append_latency = None

for i, line in enumerate(lines):
    line = line.strip()
    # Detect computation time
    if line.startswith("results for computation time"):
        match = re.search(r"computation time (\d+) us", line)
        if match:
            current_time = int(match.group(1))
    # Extract append latency (first "latency (us)" mean value)
    elif "statistic/metric, latency (us)" in line:
        append_line = lines[i + 1]  # Look at the next line
        append_match = re.match(r"mean,\s*([\d.]+)", append_line)
        if append_match:
            append_latency = float(append_match.group(1))
    # Extract other latencies (mean values)
    elif line.startswith("mean,") and current_time is not None:
        parts = line.split(",")
        if len(parts) == 6:  # Ensure correct format
            delivery_latency, compute_latency, confirm_latency, e2e_latency, queueing_delay = map(float, parts[1:])
            results.append({
                "computation_time": current_time,
                "append_latency": append_latency,
                "delivery_latency": delivery_latency,
                "compute_latency": compute_latency,
                "confirm_latency": confirm_latency,
                "e2e_latency": e2e_latency,
                "queueing_delay": queueing_delay,
            })

# Create a DataFrame
df = pd.DataFrame(results)

df.to_csv("output.csv", index=False)

# Print the CSV content
print(df.to_csv(index=False))

### Analyze order server log from the reconfiguration experiment

In [ ]:
import re
from datetime import datetime

def analyze_reconfig_log(path):
    # Sample log data (you can replace this with the contents of your log file)
    with open(path, 'r') as f:
        log_data = f.read()

    tput = []
    for line in log_data.splitlines():
        if "[real-time tput]:" in line:
            tput.append(int(line.split("ops/sec")[0].split()[-1]))

    
    return tput


In [ ]:
tputs = analyze_reconfig_log("../results/reconfig_1000/order-0.log")
# tputs = tputs[::2]

In [ ]:
import re
import matplotlib.pyplot as plt
from datetime import datetime

# File path
log_file = "../results/reconfig_1000/order-0.log"

# Regex patterns
tput_pattern = r"\[real-time tput\]: (\d+) ops/sec"
timestamp_pattern = r"(\d{2}:\d{2}:\d{2}\.\d{6})"

# Data storage
timestamps = []
tput_values = []
shard_join_request = []
shard_notified_to_be_added = []
shard_notified_to_be_removed = []
first_cut_committed = []
last_cut_committed = []
shard_leave_request = []

# Parse the log file
with open(log_file, "r") as f:
    for line in f:
        # Extract real-time throughput
        tput_match = re.search(tput_pattern, line)
        timestamp_match = re.search(timestamp_pattern, line)
        
        if tput_match and timestamp_match:
            tput_values.append(int(tput_match.group(1)))
            timestamps.append(datetime.strptime(timestamp_match.group(1), "%H:%M:%S.%f"))
        
        # Extract events for annotation
        if "Shard 1 to be added in next avl window" in line:
            shard_join_request.append(timestamp_match.group(1))

        if shard_notified_to_be_added == [] and "shardQuotas:<key:2" in line:
            shard_notified_to_be_added.append(timestamp_match.group(1))
        
        if first_cut_committed == [] and "cut:<key:2" in line:
            first_cut_committed.append(timestamp_match.group(1))
        
        if last_cut_committed == [] and "Replica 2 finalized" in line:
            last_cut_committed.append(timestamp_match.group(1))

        if "Shard 1 to be finalized in next avl window" in line:
            shard_leave_request.append(timestamp_match.group(1))

        if "Incrementing view ID because shardFinalized: true" in line:
            shard_notified_to_be_removed.append(timestamp_match.group(1))


# Convert timestamps to seconds since the start
start_time = timestamps[0]
time_in_seconds = [(ts - start_time).total_seconds() for ts in timestamps]

# Plot the data
plt.figure(figsize=(10, 6))
plt.plot(time_in_seconds, tput_values, label="Throughput (ops/sec)", color="blue")
plt.xlabel("Time (seconds)")
plt.ylabel("Throughput (ops/sec)")
plt.title("Real-time Throughput vs Time")
plt.grid()

# Event times
shard_join_request = (datetime.strptime(shard_join_request[0], "%H:%M:%S.%f") - start_time).total_seconds()
shard_notified_to_be_added_time = (datetime.strptime(shard_notified_to_be_added[0], "%H:%M:%S.%f") - start_time).total_seconds()
first_cut_committed_time = (datetime.strptime(first_cut_committed[0], "%H:%M:%S.%f") - start_time).total_seconds()
shard_leave_request_time = (datetime.strptime(shard_leave_request[0], "%H:%M:%S.%f") - start_time).total_seconds()
shard_notified_to_be_removed_time = (datetime.strptime(shard_notified_to_be_removed[0], "%H:%M:%S.%f") - start_time).total_seconds()
last_cut_committed_time = (datetime.strptime(last_cut_committed[0], "%H:%M:%S.%f") - start_time).total_seconds()

# Add vertical lines for events
plt.axvline(shard_join_request, color="red", linestyle="--", alpha=0.7, label="shard requests to join")
plt.axvline(shard_notified_to_be_added_time, color="green", linestyle="--", alpha=0.7, label="shard notified about addition window")
plt.axvline(first_cut_committed_time, color="purple", linestyle="--", alpha=0.7, label="first cut committed from new shard")
plt.axvline(shard_leave_request_time, color="orange", linestyle="--", alpha=0.7, label="shard requests to leave")
plt.axvline(shard_notified_to_be_removed_time, color="brown", linestyle="--", alpha=0.7, label="shard notified about removal window")
plt.axvline(last_cut_committed_time, color="black", linestyle="--", alpha=0.7, label="last cut committed from leaving shard")

# Zoom in to the relevant range (adjust as needed)
# plt.xlim(22.8, 23.5)
# plt.xlim(10, 65)
# plt.xlim(51, 55)

# Add legend outside the plot area
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title="Events")

# Adjust layout to ensure the legend doesn't overlap with the plot
plt.tight_layout()

# Save the plot
plt.savefig("adding_reconfig_throughput_annotated.png", dpi=600)

plt.show()


## Analyze script for lagfix expt

In [ ]:
import re
import math

def find_nearest_multiple_of_3(number):
    return math.ceil(number / 3) * 3
    
def process_logs(logfile1, logfile2, logfile3):
    # Step 1: Extract the number from logfile1
    with open(logfile1, 'r') as file1:
        for line in file1:
            match = re.search(r"burst record with lsn (\d+)", line)
            if match:
                number = int(match.group(1))
                y = find_nearest_multiple_of_3(number)
                break
        else:
            raise ValueError("No 'burst record with lsn' found in logfile1")
    
    # Step 2: Locate and fetch lines from logfile2
    with open(logfile2, 'r') as file2:
        lines = file2.readlines()
    
    target_index = None
    for i, line in enumerate(lines):
        if f"cut:{y} cut:0" in line:
            target_index = i
            break
    
    if target_index is None:
        raise ValueError(f"No line containing 'cut:{y} cut:0' found in logfile2")
    
    # Fetch 2000 lines before and after
    start = max(0, target_index - 200)
    end = min(len(lines), target_index + 200)
    selected_lines = lines[start:end]
    
    # Step 3: Write to logfile3
    with open(logfile3, 'w') as file3:
        file3.writelines(selected_lines)

    return y



In [ ]:
logfile1 = "../results/lagfix/data-0-0.log"
logfile2 = "../results/lagfix/order-0.log"
logfile3 = "output.log"

y = process_logs(logfile1, logfile2, logfile3)

In [ ]:
import re

def extract_events(log_file, y):
    shard_0_cuts = []
    shard_1_cuts = []
    
    lagfix_detected = []
    lagfix_notified = []
    lagfix_resolved = []
    burst_start = []

    # Generic pattern for timestamps
    timestamp_pattern = r"(\d{2}:\d{2}:\d{2}\.\d{6})"

    with open(log_file, 'r') as file:
        for line in file:
            # Check for shard 0 cuts
            match_shard_0 = re.search(fr"{timestamp_pattern} cut:(\d+) cut:0", line)
            if match_shard_0:
                timestamp, cut = match_shard_0.groups()
                shard_0_cuts.append((timestamp, int(cut)))

            # Check for shard 1 cuts
            match_shard_1 = re.search(fr"{timestamp_pattern} localReplicaID:1 cut:0 cut:(\d+)", line)
            if match_shard_1:
                timestamp, cut = match_shard_1.groups()
                shard_1_cuts.append((timestamp, int(cut)))

            if burst_start == [] and "cut:" + str(y) + " cut:0" in line:
                burst_start.append(re.search(timestamp_pattern, line).group(1))

            if burst_start != [] and lagfix_detected == [] and "significant lag in cuts:" in line:
                lagfix_detected.append(re.search(timestamp_pattern, line).group(1))

            if burst_start != [] and lagfix_notified == [] and "adjustmentSignal:" in line:
                lagfix_notified.append(re.search(timestamp_pattern, line).group(1))
            
            if burst_start != [] and lagfix_resolved == [] and "fixedLag:true" in line:
                lagfix_resolved.append(re.search(timestamp_pattern, line).group(1))


    return shard_0_cuts, shard_1_cuts, lagfix_detected, lagfix_notified, lagfix_resolved, burst_start


# Example usage
log_file = "output.log"
shard_0_cuts, shard_1_cuts, lagfix_detected, lagfix_notified, lagfix_resolved, burst_start = extract_events(log_file, y)




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator, MaxNLocator


def parse_timestamp(timestamp):
    return datetime.strptime(timestamp, "%H:%M:%S.%f")


def plot_events_zoomed(
    shard_0_cuts, shard_1_cuts, lagfix_detected, lagfix_notified, lagfix_resolved, burst_start, zoom_event="Lag Detected", zoom_window_ms=100
):
    # Convert timestamps to datetime objects
    shard_0_times = [parse_timestamp(t) for t, _ in shard_0_cuts]
    shard_0_values = [1 for _ in shard_0_cuts]  # Level 1 for Shard 0

    shard_1_times = [parse_timestamp(t) for t, _ in shard_1_cuts]
    shard_1_values = [2 for _ in shard_1_cuts]  # Level 2 for Shard 1

    # Parse event timestamps
    event_times = {
        "Lag Detected": [parse_timestamp(lagfix_detected[0])] if lagfix_detected else [],
        "Lag Notified": [parse_timestamp(lagfix_notified[0])] if lagfix_notified else [],
        "Lag Resolved": [parse_timestamp(lagfix_resolved[0])] if lagfix_resolved else [],
        "Burst Start": [parse_timestamp(burst_start[0])] if burst_start else [],
    }

    # Event colors for differentiation
    event_colors = {
        "Lag Detected": "red",
        "Lag Notified": "purple",
        "Lag Resolved": "green",
        "Burst Start": "brown",
    }

    # Determine the time range for zooming
    if zoom_event not in event_times or not event_times[zoom_event]:
        print(f"No event found for {zoom_event}.")
        return

    zoom_time = event_times[zoom_event][0]
    zoom_start = zoom_time - timedelta(milliseconds=zoom_window_ms)
    zoom_end = zoom_time + timedelta(milliseconds=zoom_window_ms)

    # Filter data for the zoomed time range
    shard_0_times_zoomed = [t for t in shard_0_times if zoom_start <= t <= zoom_end]
    shard_1_times_zoomed = [t for t in shard_1_times if zoom_start <= t <= zoom_end]

    # Start plotting
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot cut points for Shard 0 with stacking
    ax.scatter(shard_0_times_zoomed, [1 + 0.05 * i for i in range(len(shard_0_times_zoomed))], label="Shard 0 Cuts", color="blue", marker="o", s=40, alpha=0.6)

    # Plot cut points for Shard 1 with stacking
    ax.scatter(shard_1_times_zoomed, [2 + 0.05 * i for i in range(len(shard_1_times_zoomed))], label="Shard 1 Cuts", color="orange", marker="o", s=40, alpha=0.6)

    # Plot vertical lines for events within the zoom window
    for event_name, times in event_times.items():
        for time in times:
            if zoom_start <= time <= zoom_end:
                ax.axvline(time, linestyle="--", label=event_name, color=event_colors[event_name], alpha=0.7)

    # Adjust x-axis for 1 ms ticks
    ax.xaxis.set_major_locator(MultipleLocator(0.001))  # 1 ms ticks
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%S.%f"))  # Show only seconds and milliseconds

    # Adjust number of ticks on the x-axis
    ax.xaxis.set_major_locator(MaxNLocator(nbins=10, prune='both'))  # Use nbins to control tick density

    # Set the x-limits to zoom in on the window
    ax.set_xlim(zoom_start, zoom_end)

    # Rotate x-axis labels for better readability
    ax.tick_params(axis='x', rotation=45)

    # Adjust y-axis
    ax.set_yticks([1, 2])
    ax.set_yticklabels(["Shard 0", "Shard 1"])

    # Add labels and legend
    ax.set_xlabel("Time (seconds.milliseconds)")
    ax.set_ylabel("Shards")
    ax.set_title(f"Zoomed View: {zoom_event} (±{zoom_window_ms} ms)")
    ax.legend(loc="upper left", bbox_to_anchor=(1, 1), title="Legend", shadow=True)

    # Show the plot
    plt.tight_layout()
    plt.savefig("output.png", dpi=600)


plot_events_zoomed(shard_0_cuts, shard_1_cuts, lagfix_detected, lagfix_notified, lagfix_resolved, burst_start, zoom_event="Lag Detected", zoom_window_ms=20)
